In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [59]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [60]:
df.shape

(60000, 785)

In [61]:
torch.manual_seed(42)

In [62]:
#check is gpu is avaible or not
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [63]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
x = df.drop(columns='label', axis=1)
y = df['label']

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [66]:
#Standardiztion
x_train = x_train/255.0
x_test = x_test/255.0

In [67]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels =  torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [68]:
train_dataset = CustomDataset(x_train.values, y_train.values)
test_dataset = CustomDataset(x_test.values, y_test.values)

In [69]:
class MyNN(nn.Module):
  def __init__(self, input_dim, output_dim, num_hidden_layer, neuron_per_layer, dropout_rate):

    super().__init__()
    layers = []

    for i in range(num_hidden_layer):
      layers.append(nn.Linear(input_dim, neuron_per_layer))
      layers.append(nn.BatchNorm1d(neuron_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neuron_per_layer
    layers.append(nn.Linear(neuron_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)

In [70]:
def objetive(trail):
  #next hyperparameter
  num_hidden_layer = trail.suggest_int("num_hiden_layer", 1, 5)
  neuron_per_layer = trail.suggest_int("neuron_per_layer", 8, 128, step=8)
  epochs           = trail.suggest_int("epochs", 1, 10)
  learning_rate    = trail.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
  dropout_rate     = trail.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size       = trail.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name   = trail.suggest_categorical("optimizer", ['Adam', 'RMSprop', 'SGD'])
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  #model pararemter
  input_dim = x_train.shape[1]
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layer, neuron_per_layer, dropout_rate).to(device) # Move model to device

  #parameter analysise
  learning_rate_fixed = 0.1 # Renamed to avoid confusion with hyperparameter
  epochs_fixed = 100 # Renamed to avoid confusion with hyperparameter

  #optimizer
  critertion = nn.CrossEntropyLoss()
  if(optimizer_name == 'Adam'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Changed learning_rate to lr
  elif(optimizer_name == 'RMSprop'):
    optimizer = optim.RMSprop(model.parameters(), lr = learning_rate) # Changed learning_rate to lr
  else:
    optimizer = optim.SGD(model.parameters(), lr = learning_rate) # Changed learning_rate to lr

  for epoch in range(epochs):
    model.train() # Set model to training mode
    for data_features, data_labels in train_loader:
      data_features = data_features.to(device)
      data_labels = data_labels.to(device) # Corrected variable name

      output = model(data_features)
      loss = critertion(output, data_labels) # Corrected typo batch_labeks

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  model.eval() # Set model to evaluation mode
  correct = 0
  total = 0
  with torch.no_grad():
    for data_features, data_labels in test_loader:
      data_features = data_features.to(device)
      data_labels = data_labels.to(device)
      outputs = model(data_features)
      _, predicted = torch.max(outputs.data, 1)
      total += data_labels.size(0)
      correct += (predicted == data_labels).sum().item()
  accuracy = correct / total
  return accuracy # Return accuracy for hyperparameter optimization

In [71]:
!pip install optuna

In [72]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objetive, n_trials=10)

[I 2025-12-14 07:17:31,494] A new study created in memory with name: no-name-e2c2187d-d3ce-4f9b-8280-0d5c8844cc4b
[I 2025-12-14 07:17:57,989] Trial 0 finished with value: 0.5085833333333334 and parameters: {'num_hiden_layer': 3, 'neuron_per_layer': 128, 'epochs': 10, 'learning_rate': 4.803061294020903e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.5085833333333334.
[I 2025-12-14 07:18:05,954] Trial 1 finished with value: 0.7946666666666666 and parameters: {'num_hiden_layer': 1, 'neuron_per_layer': 24, 'epochs': 4, 'learning_rate': 4.9680144349996364e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7946666666666666.
[I 2025-12-14 07:18:32,113] Trial 2 finished with value: 0.87375 and parameters: {'num_hiden_layer': 3, 'neuron_per_layer': 72, 'epochs': 8, 'learning_rate': 6.371197997032686e-05, 'dropout_rate': 0.1, 'batch_size': 64, 'optimizer': 'Adam'}. Best i

In [73]:
study.best_params

{'num_hiden_layer': 3,
 'neuron_per_layer': 72,
 'epochs': 8,
 'learning_rate': 6.371197997032686e-05,
 'dropout_rate': 0.1,
 'batch_size': 64,
 'optimizer': 'Adam'}

In [74]:
study.best_value

0.87375